In [2]:
# ============================================
# 1. 입력 JSON / 출력 JSON 경로 지정
# ============================================
INPUT_TSV = "/Users/taeyoonkwack/Documents/compliance_checking/environments/GDPR-13/data/original/data.tsv"      # 원본 json 파일
OUTPUT_TSV = "/Users/taeyoonkwack/Documents/compliance_checking/environments/GDPR-13/data/original/sample.tsv"   # 샘플링된 json 파일 저장 위치

# ============================================
# 2. 데이터 로드 및 200개 샘플링
# ============================================
import pandas as pd

# 데이터 로드
df = pd.read_csv(INPUT_TSV, sep="\t")

# label 컬럼 확인
if "label" not in df.columns:
    raise ValueError("입력 TSV에 'label' 컬럼이 없습니다.")

# 이진화: label == 0 → 0, label != 0 → 1
df["_bin"] = (df["label"].astype(str).str.strip() != "0").astype(int)

# 클래스별 분리
pos = df[df["_bin"] == 1]
neg = df[df["_bin"] == 0]

# 충분한 데이터 확인
if len(pos) < 100 or len(neg) < 100:
    raise ValueError(f"데이터가 부족합니다. pos={len(pos)}, neg={len(neg)}")

# 샘플링 (재현성을 위해 random_state 고정)
pos_s = pos.sample(n=100, random_state=42)
neg_s = neg.sample(n=100, random_state=42)

# 합치고 섞기
sample = pd.concat([pos_s, neg_s], axis=0).sample(frac=1.0, random_state=42).drop(columns=["_bin"])

# 저장
sample.to_csv(OUTPUT_TSV, sep="\t", index=False)

print(f"✅ 샘플 저장 완료: {OUTPUT_TSV}")
print("라벨 분포:\n", sample["label"].astype(str).str.strip().value_counts())


✅ 샘플 저장 완료: /Users/taeyoonkwack/Documents/compliance_checking/environments/GDPR-13/data/original/sample.tsv
라벨 분포:
 label
0    100
1     34
3     32
6     12
2      7
4      7
8      5
5      2
9      1
Name: count, dtype: int64
